In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Button, HBox, Output
from amplify import Solver, BinarySymbolGenerator, BinaryQuadraticModel
from amplify.client import FixstarsClient
from amplify.constraint import one_hot

client = FixstarsClient()

%matplotlib inline

# 数独

数独は代表的な制約充足問題の一つで、以下の[ルール](https://www.nikoli.co.jp/ja/puzzles/sudoku/)に従って数字を埋めていくペンシルパズルです。

* 空いているマスに1〜9のいずれかの数字を入れる
* 縦・横の各列・各行と3×3のブロック内に同じ数字が複数入ってはいけない

制約充足問題はグラフ彩色問題と同様にスケジューリング問題等の定式化で現れます。

次のデモは 9x9 の数独の問題（引用元: [数独問題集-2018年3月1日-最高級-](http://www.sudokugame.org/archive/printable.php?nd=4&y=2018&m=03&d=1)）を QUBO 定式化を用いて Amplify Annealing Engine で実行します。  
「Run」ボタンをクリックすると出力された解が表示されます。

In [ ]:
from math import sqrt

sudoku_initial = np.array(
    [
        [2, 0, 5, 1, 3, 0, 0, 0, 4],
        [0, 0, 0, 0, 4, 8, 0, 0, 0],
        [0, 0, 0, 0, 0, 7, 0, 2, 0],
        [0, 3, 8, 5, 0, 0, 0, 9, 2],
        [0, 0, 0, 0, 9, 0, 7, 0, 0],
        [0, 0, 0, 0, 0, 0, 4, 5, 0],
        [8, 6, 0, 9, 7, 0, 0, 0, 0],
        [9, 5, 0, 0, 0, 0, 0, 3, 1],
        [0, 0, 4, 0, 0, 0, 0, 0, 0],
    ]
)


def print_sudoku(sudoku):

    N = len(sudoku)
    n = int(sqrt(N))
    width = len(str(N))
    for i in range(N):
        line = ""
        if i % n == 0 and i != 0:
            print("-" * ((width + 1) * n * n + 2 * (n - 1) - 1))
        for j in range(len(sudoku[i])):
            if j % n == 0 and j != 0:
                line += "| "
            line += (str(sudoku[i][j]).rjust(width) if sudoku[i][j] != 0 else " ") + " "
        print(line)


def sudoku_solve(*args):

    N = len(sudoku_initial)
    n = int(sqrt(N))
    q = BinarySymbolGenerator().array(N, N, N)

    for i, j in zip(*np.where(sudoku_initial != 0)):
        k = sudoku_initial[i, j] - 1

        q[i, :, k] = 0  # 制約(a)
        q[:, j, k] = 0  # 制約(b)
        q[i, j, :] = 0  # 制約(d)
        for m in range(N):
            q[(n * (i // n) + m // n), (n * (j // n) + m % n), k] = 0  # 制約(c)

        q[i, j, k] = 1  # 変数の値を1に指定する

    # (a): 各行には同じ数字が入らない制約条件
    row_constraints = [one_hot(q[i, :, k]) for i in range(N) for k in range(N)]

    # (b): 各列には同じ数字が入らない制約条件
    col_constraints = [one_hot(q[:, j, k]) for j in range(N) for k in range(N)]

    # (d): 一つのマスには一つの数字しか入らない制約条件
    num_constraints = [one_hot(q[i, j, :]) for i in range(N) for j in range(N)]

    # (c): nxnブロック内には同じ数字が入らない制約条件
    block_constraints = [
        one_hot(sum([q[i + m // n, j + m % n, k] for m in range(N)]))
        for i in range(0, N, n)
        for j in range(0, N, n)
        for k in range(N)
    ]

    constraints = (
        sum(row_constraints)
        + sum(col_constraints)
        + sum(num_constraints)
        + sum(block_constraints)
    )

    solver = Solver(client)
    solver.client.parameters.timeout = 1000
    model = BinaryQuadraticModel(constraints)
    result = solver.solve(model)
    i = 0
    while len(result) == 0:
        if i > 5:
            raise RuntimeError()
        result = solver.solve(model)
        i += 1

    values = result[0].values
    q_values = q.decode(values)
    answer = np.array([np.where(np.array(q_values[i]) != 0)[1] + 1 for i in range(N)])
    print_sudoku(answer)

In [ ]:
%%html
<style>.sudoku {
    width: 100%;
    margin: 0 auto;
    font-size: 150%;
    font-size: 1.vw;
}

In [ ]:
sudoku_run_btn = Button(description="Run", button_style="", tooltip="Run", icon="check")
sudoku_problem_out = Output()
sudoku_problem_out.add_class("sudoku")
sudoku_result_out = Output()
sudoku_result_out.add_class("sudoku")


def show_sudoku_problem():
    with sudoku_problem_out:
        print_sudoku(sudoku_initial)


def show_sudoku_result(btn):
    with sudoku_result_out:
        sudoku_result_out.clear_output()
        sudoku_solve()


sudoku_run_btn.on_click(show_sudoku_result)
display(HBox([sudoku_run_btn]), HBox([sudoku_problem_out, sudoku_result_out]))
show_sudoku_problem()